In [254]:
import numpy as np
import pandas as pd
import os
import re

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
init_notebook_mode(connected = True)

from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import PorterStemmer
from nltk import WordNetLemmatizer

In [61]:
from plotly import __version__

In [255]:
#Get all csv files in current directory
get_filelist = [f for f in os.listdir(os.curdir) if f.endswith(".csv")]

In [256]:
files = get_filelist[:-2]

In [257]:
files

['0_999.csv',
 '1000_2999.csv',
 '12000_13999.csv',
 '14000_15999.csv',
 '16000_17999.csv',
 '18000_19999.csv',
 '20000_22999.csv',
 '23000_25999.csv',
 '26000_28999.csv',
 '29000_31999.csv',
 '3000_5999.csv',
 '32000_34999.csv',
 '6000_8999.csv',
 '9000_11999.csv']

In [258]:
df = pd.concat([pd.read_csv(f) for f in files])

In [150]:
for c in df:
    print(c, '\n', 'uniques:', df[c].nunique(), '\n', ('*'*40))

Title 
 uniques: 5437 
 ****************************************
Location 
 uniques: 788 
 ****************************************
Company 
 uniques: 2613 
 ****************************************
Salary 
 uniques: 421 
 ****************************************
Summary 
 uniques: 6568 
 ****************************************
Description 
 uniques: 13712 
 ****************************************
Review 
 uniques: 1498 
 ****************************************
benefit_rating 
 uniques: 62 
 ****************************************
comp_rating_overall 
 uniques: 33 
 ****************************************
culture_rating 
 uniques: 68 
 ****************************************
job_description 
 uniques: 3357 
 ****************************************
jsecurity_rating 
 uniques: 69 
 ****************************************
mgmt_rating 
 uniques: 70 
 ****************************************
wl_bal_rating 
 uniques: 64 
 ****************************************


In [259]:
df.reset_index(inplace=True, drop=True)

In [260]:
df.head(3)

,Title,Location,Company,Salary,Summary,Description,Review,benefit_rating,comp_rating_overall,culture_rating,job_description,jsecurity_rating,mgmt_rating,wl_bal_rating
0,External Research Coordinator,NaN,Moffitt Cancer Center,NaN,Conducts monthly q...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Moffitt-Cancer-Center,3.8,4.0,3.8,Moffitt Cancer Center is internationally recog...,3.6,3.5,3.9
1,Information Security Lead Data Scientist - VP ...,NaN,Citi,NaN,Serve as a Lead Da...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Citi,3.9,3.9,3.8,"Primary Location: United States,Florida,Tampa ...",3.5,3.6,3.8
2,Process Engineer,NaN,James Hardie,NaN,Your decision-maki...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/James-Hardie-Buildi...,3.9,3.5,3.2,The Process Engineer manages the on-boarding a...,3.2,3.0,3.1


In [261]:
df.loc[df['Title'] == 'None', col] = None
df.loc[df['Company'] == 'None', col] = None
df.loc[df['Description'] == 'None', col] = None
df.loc[df['Review'] == 'None', col] = None
df.loc[df['job_description'] == 'None', col] = None

In [262]:
#find empty values
np.sum(df.isnull())

Title                      0
Location                9709
Company                    0
Salary                 32253
Summary                    1
Description                0
Review                     0
benefit_rating          7820
comp_rating_overall     7914
culture_rating          7820
job_description            0
jsecurity_rating        7820
mgmt_rating             7820
wl_bal_rating           7820
dtype: int64

In [263]:
df = df.loc[df['Location'].notnull()]

In [264]:
df.shape

(25292, 14)

In [157]:
#replace all empty spaces by one empty space
df['job_description'] = df['job_description'].apply(lambda x: re.sub('\s+', ' ', x))

In [133]:
languages_re={'R':'(?<=[\W])R(?=[\W])', 'python':'python', 'Java': 'java', 'Ruby':'ruby','Bash':'bash',\
              'vba':'vba', 'css':'css', 'Javascript':"JAVA-?\s?SCRIPT", 'Scala': 'scala','C':'(?<=[\W])C(?=[\W])',\
             'HTML':'html', 'Perl':'perl', 'SQL':'sql'}

databases_re={ 'Oracle':'oracle', 'MongoDB':'mongodb', 'MySQL':'mysql', 'PostgreSQL':'postgresql', \
             'SQL Server':'sql server', 'Cassandra':'cassandra'}


infrastructures_re={'Hadoop': 'hadoop', 'Spark':'spark', 'AWS':'AWS|[Aa]mazon [Ww]eb [Ss]ervice', 'Hive':'hive',\
                    'Pig':'pig'}

data_science_packages_re={'SAS':'SAS','Matlab':'matlab', 'Tableau':'tableau','TensorFlow':'tensor( ?)flow',\
                          'Excel':'(?<=[\W])Excel(?=[\W])','Stata': 'stata','H2O':'h2o', 'BI':'BI',\
                          'Scikit Learn':'scikit'}

os_re={'Linux': 'linux','windows':'windows', 'mac':'mac '}


skill_re = {'Machine Learning':'ML|[Mm]achine [lL]earning', 'Data Visualization':'visualization',\
            'Deep Learning':'[Dd]eep [Ll]earning','AI':'AI|[Aa]rtificial [Ii]nteligence',\
            'Neural Network':'[Nn]eural ?[Nn]','NLP':'[Nn]atural [Ll]anguage [Pp]rocessing|NLP',\
            'Predictive Modeling':'[Pp]redictive [Mm]odel', 'Web Scraping':'([Ww]eb )?[Ss]crap',\
            'Descision Tree':'[Tt]ree', 'Clustering':'[[Cc]luster' }

tool_re_case_sensitive=['R','SAS', 'C', 'SPSS', 'BI', 'C/C++','Data Visualization','Web Scraping']


all_tools_re = {'Languages':languages_re, 'Databases': databases_re, 'Data Science Tools':data_science_packages_re,\
                'OSes':os_re, 'Infrastructure':infrastructures_re, 'Data Science Methods':skill_re }

skill_groups_df={}

for tool_name, my_df in all_tools_re.items():
    for tool in my_df:
        df[tool]= df.job_description.str.contains(my_df[tool], case = (tool in tool_re_case_sensitive or tool_name=='Data Science Methods' ))
    
    skill_groups_df[tool_name] = (pd.DataFrame(df.loc[:,my_df.keys()].apply(pd.value_counts).loc[True].sort_values(ascending=False)))

C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [180]:
#skill_groups_df

In [183]:
l = ['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Charlotte', 'Chicago', 'Cincinnati', 'Columbus', 
          'Dallas', 'Denver', 'Houston', 'Indianapolis', 'Jacksonville', 'Kansas+City', 'Los+AngelesNew+York', 
          'Miami', 'Minneapolis', 'Nashville', 'Oakland', 'Philadelphia', 'Phoenix', 'Pittsburgh', 'Portland', 
          'San+Diego', 'San+Francisco', 'San+Jose', 'Seattle', 'St.+Louis', 'Tampa', 'Washington%2C+DC']

In [199]:
l_dict = {'cities': ['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Charlotte', 'Chicago', 'Cincinnati', 'Columbus', 
          'Dallas', 'Denver', 'Houston', 'Indianapolis', 'Jacksonville', 'Kansas+City', 'Los+AngelesNew+York', 
          'Miami', 'Minneapolis', 'Nashville', 'Oakland', 'Philadelphia', 'Phoenix', 'Pittsburgh', 'Portland', 
          'San+Diego', 'San+Francisco', 'San+Jose', 'Seattle', 'St.+Louis', 'Tampa', 'Washington%2C+DC']}

In [265]:
df1 = df.copy()

In [266]:
df1.shape

(25292, 14)

In [201]:
for a,b in l_dict.items():
    for loc in b: 
        df[loc] = df.Location.str.contains(b[loc])

#df1.Location.str.contains(any(l))

TypeError: list indices must be integers or slices, not str

In [214]:
for a in df1['Company']:
    if any(s in a for s in l):
        print('yeah')

yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah


In [227]:
if any(s in n for s in l):
    print(l[s])

NameError: name 's' is not defined

In [244]:
(re.search('Atlanta', 'Altantava'))

In [247]:
b = ['xxx', 'yyy']
for a in b:
    for c in df1['Location']:
        if(bool(re.search(a,c))):
                print('y')

In [268]:
for a in l:
    for i in range(len(df1)):
        for c in df1.iloc[i]['Location']:
            if(bool(re.search(a,c))):
                df1.iloc[i]['location'] = a

KeyError: 'location'

0
1
2
3
4


In [253]:
df1['Location']

3       Atlanta
4       Atlanta
5       Atlanta
6       Atlanta
7       Atlanta
8       Atlanta
9       Atlanta
10      Atlanta
11      Atlanta
12      Atlanta
17      Atlanta
18      Atlanta
19      Atlanta
20      Atlanta
21      Atlanta
22      Atlanta
23      Atlanta
24      Atlanta
25      Atlanta
26      Atlanta
32      Atlanta
33      Atlanta
34      Atlanta
35      Atlanta
36      Atlanta
37      Atlanta
38      Atlanta
39      Atlanta
40      Atlanta
41      Atlanta
         ...   
2961    Atlanta
2965    Atlanta
2966    Atlanta
2967    Atlanta
2968    Atlanta
2969    Atlanta
2970    Atlanta
2971    Atlanta
2972    Atlanta
2973    Atlanta
2974    Atlanta
2978    Atlanta
2979    Atlanta
2980    Atlanta
2981    Atlanta
2982    Atlanta
2983    Atlanta
2984    Atlanta
2985    Atlanta
2986    Atlanta
2987    Atlanta
2991    Atlanta
2992    Atlanta
2993    Atlanta
2994    Atlanta
2995    Atlanta
2996    Atlanta
2997    Atlanta
2998    Atlanta
2999    Atlanta
Name: Location, Length: 

In [250]:
df1['Location']

3       Atlanta
4       Atlanta
5       Atlanta
6       Atlanta
7       Atlanta
8       Atlanta
9       Atlanta
10      Atlanta
11      Atlanta
12      Atlanta
17      Atlanta
18      Atlanta
19      Atlanta
20      Atlanta
21      Atlanta
22      Atlanta
23      Atlanta
24      Atlanta
25      Atlanta
26      Atlanta
32      Atlanta
33      Atlanta
34      Atlanta
35      Atlanta
36      Atlanta
37      Atlanta
38      Atlanta
39      Atlanta
40      Atlanta
41      Atlanta
         ...   
2961    Atlanta
2965    Atlanta
2966    Atlanta
2967    Atlanta
2968    Atlanta
2969    Atlanta
2970    Atlanta
2971    Atlanta
2972    Atlanta
2973    Atlanta
2974    Atlanta
2978    Atlanta
2979    Atlanta
2980    Atlanta
2981    Atlanta
2982    Atlanta
2983    Atlanta
2984    Atlanta
2985    Atlanta
2986    Atlanta
2987    Atlanta
2991    Atlanta
2992    Atlanta
2993    Atlanta
2994    Atlanta
2995    Atlanta
2996    Atlanta
2997    Atlanta
2998    Atlanta
2999    Atlanta
Name: Location, Length: 

In [186]:
df.Location.str.contains('Atlanta')

0         NaN
1         NaN
2         NaN
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13        NaN
14        NaN
15        NaN
16        NaN
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27        NaN
28        NaN
29        NaN
        ...  
2970    False
2971    False
2972    False
2973    False
2974    False
2975      NaN
2976      NaN
2977      NaN
2978    False
2979    False
2980    False
2981    False
2982    False
2983    False
2984    False
2985    False
2986    False
2987    False
2988      NaN
2989      NaN
2990      NaN
2991    False
2992    False
2993    False
2994    False
2995    False
2996    False
2997    False
2998    False
2999    False
Name: Location, Length: 23001, dtype: object

In [167]:
df['Location']

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                               Tampa, FL
4                                               Tampa, FL
5                                               Tampa, FL
6                                               Tampa, FL
7                                               Tampa, FL
8                                         Tampa, FL 33612
9                                         Tampa, FL 33625
10                                              Tampa, FL
11                                              Tampa, FL
12                                              Tampa, FL
13                                                    NaN
14                                                    NaN
15                                                    NaN
16                                                    NaN
17            

In [ ]:
df_more = df_more[df_more.Salary != 'None'].drop_duplicates().dropna()

In [ ]:
df_more = df_more[df_more.Salary.str.contains("hour") == False]
df_more = df_more[df_more.Salary.str.contains("month") == False]

In [ ]:
def get_mean_salary(df, col):
    df[col].replace({'\$':''}, regex = True, inplace = True)
    df[col].replace(regex=True,inplace=True,to_replace=r'\D',value=r' ')
    df[col] = df[col].str.strip()
    df[col] = df_more[col].str.replace('   ',',')
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'min', 1:'max'}))
    df['min'] = df['min'].str.replace(' ','')
    df['min'] = df['min'].astype('float64')
    df['max'] = df['max'].str.replace(' ','')
    df['max'] = df['max'].astype('float64')
    df.drop(['Salary'], axis = 1, inplace = True)
    df['Salary'] = df[['min', 'max']].mean(axis=1)
    df.drop(['min','max'], axis=1, inplace = True)
    return df

def get_city_state(df,col):
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    df['State'] = [x[0:3] for x in df['State'] if x != None]
    df.drop(['Location'], axis = 1, inplace = True)
    return df


In [ ]:
df_more = get_mean_salary(df_more, 'Salary')
df_more = get_city_state(df_more, 'Location')